In [ ]:
import tifffile as tiff
import os
import glob
import matplotlib.pyplot as plt
train_img_path = "C:/github/AISparks/ai_spark_oaz/jiwoo/data/image/train_img"
train_mask_path = "C:/github/AISparks/ai_spark_oaz/jiwoo/data/image/train_mask"
train_img_pattern = os.path.join(train_img_path, "*.tif")
train_mask_pattern = os.path.join(train_mask_path, "*.tif")
img_files = glob.glob(train_img_pattern)
mask_files = glob.glob(train_mask_pattern)

In [ ]:
import tifffile as tiff
import numpy as np
import matplotlib.pyplot as plt

# Read the MRI stack TIFF file
train_img = tiff.imread('./data/image/train_img/train_img_0.tif')
mask_img = tiff.imread('./data/image/train_mask/train_mask_0.tif')
'''max_val=0
for i in range(256):
    for j in range(256):
        for k in range(10):
            if train_img[i, j, k] > max_val:
                max_val = train_img[i, j, k]
                print(i, j, k, train_img[i, j, k])'''
'''
#print(train_img)
print('Image dimensions:', train_img.shape)
a,b= mask_img.shape
for i in range(a):
    for j in range(b):
        if mask_img[i,j]!=0:
            print(i, j, mask_img[i,j])
# Define the number of channels
num_channels = train_img.shape[2]

# Create a new figure for the subplots
fig, axs = plt.subplots(3, 5, figsize=(18, 12))

# Iterate over each channel and plot its slice
for i in range(num_channels):
    if i < 5:  # Display first 5 channels in the first row
        axs[0, i].imshow(train_img[:, :, i], cmap='gray')
        axs[0, i].set_title('Channel {}'.format(i))
        axs[0, i].axis('off')
    elif i < 10:  # Display next 5 channels in the second row
        axs[1, i-5].imshow(train_img[:, :, i], cmap='gray')
        axs[1, i-5].set_title('Channel {}'.format(i))
        axs[1, i-5].axis('off')

# Display the mask image as a single large image in the third row
axs[2, 0].imshow(mask_img, cmap='gray')
axs[2, 0].set_title('Mask Image')
axs[2, 0].axis('off')

# Remove any remaining empty subplots
for i in range(1, 5):
    axs[2, i].remove()

plt.tight_layout()
plt.show()'''

In [ ]:
import os
import rasterio
import csv

# Define the path to your images
image_folder = './data/image/train_img/'
output_csv = './data/image/train_img_info.csv'

# Function to count the number of TIFF files in the folder
def count_files(folder):
    count = 0
    for file in os.listdir(folder):
        if file.endswith('.tif'):
            count += 1
    return count

# Function to extract information from each TIFF file
def extract_info(file_path):
    with rasterio.open(file_path) as dataset:
        profile = dataset.profile
        driver = profile['driver']
        dtype = profile['dtype']
        nodata = profile['nodata']
        width = profile['width']
        height = profile['height']
        count = profile['count']
        crs = int(profile['crs'].to_epsg())  # Extracting the EPSG code
        transform = profile['transform']
        blockysize = profile['blockysize']
        tiled = profile['tiled']
        interleave = profile['interleave']
        bounds= dataset.bounds
        bounds_info = (bounds.left, bounds.bottom, bounds.right, bounds.top)
        print(bounds_info)
        lnglat = dataset.lnglat()
        res = dataset.res
        return [driver, dtype, nodata, width, height, count, crs, transform, blockysize, tiled, interleave, bounds_info, lnglat, res]

# Get the total number of TIFF files in the folder
total_files = count_files(image_folder)

# Write information to CSV
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write header row
    writer.writerow(['Filename', 'Driver', 'Dtype', 'Nodata', 'Width', 'Height', 'Count', 'CRS', 'Transform', 'Blockysize', 'Tiled', 'Interleave', 'Bounding Box (Left, Bottom, Right, Top)', 'LngLat', 'Resolution'])
    # Loop through each TIFF file
    for file_name in os.listdir(image_folder):
        if file_name.endswith('.tif'):
            file_path = os.path.join(image_folder, file_name)
            info = [file_name] + extract_info(file_path)
            writer.writerow(info)

print("CSV file generated successfully.")


In [ ]:
import os
import rasterio
import csv

# Define the paths to your images
image_folder = './data/image/train_img/'
mask_folder = './data/image/train_mask/'
output_csv = './data/image/train_img_info_wtmask.csv'

# Function to count the number of TIFF files in the folder
def count_files(folder):
    count = 0
    for file in os.listdir(folder):
        if file.endswith('.tif'):
            count += 1
    return count

# Function to extract information from each TIFF file
def extract_info(file_path, mask_path):
    with rasterio.open(file_path) as dataset:
        profile = dataset.profile
        driver = profile['driver']
        dtype = profile['dtype']
        nodata = profile['nodata']
        width = profile['width']
        height = profile['height']
        count = profile['count']
        crs = int(profile['crs'].to_epsg())  # Extracting the EPSG code
        transform = profile['transform']
        blockysize = profile['blockysize']
        tiled = profile['tiled']
        interleave = profile['interleave']
        bounds = dataset.bounds
        bounds_info = (bounds.left, bounds.bottom, bounds.right, bounds.top)
        lnglat = dataset.lnglat()
        res = dataset.res
        
        # Open corresponding mask image and count pixels with value 1
        with rasterio.open(mask_path) as mask_dataset:
            mask_data = mask_dataset.read(1)  # Read the first band
            mask_pixels = (mask_data == 1).sum()  # Count pixels with value 1
        
        return [driver, dtype, nodata, width, height, count, crs, transform, blockysize, tiled, interleave, bounds_info, lnglat, res, mask_pixels]

# Get the total number of TIFF files in the folder
total_files = count_files(image_folder)


# Write information to CSV
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write header row
    writer.writerow(['Filename', 'Driver', 'Dtype', 'Nodata', 'Width', 'Height', 'Count', 'CRS', 'Transform', 'Blockysize', 'Tiled', 'Interleave', 'Bounding Box (Left, Bottom, Right, Top)', 'LngLat', 'Resolution', 'Mask Pixels'])
    # Loop through each TIFF file
    for file_index in range(total_files):
        print(file_index)
        image_name= f'train_img_{file_index}.tif'
        mask_name= f'train_mask_{file_index}.tif'
        file_path = os.path.join(image_folder, image_name)
        mask_path = os.path.join(mask_folder, mask_name) # Constructing mask file path
        info = [image_name] + extract_info(file_path, mask_path)
        writer.writerow(info)

print("CSV file generated successfully.")


In [ ]:
mask_img = tiff.imread('./data/image/train_mask/train_mask_30145.tif')
count=0
for i in range(256):
    for j in range(256):
            if mask_img[i, j] !=0:
                count+=1
                print(i, j, mask_img[i, j])
print(count)


In [ ]:
# Define the paths to your images
import random
import csv
import pandas as pd

image_folder = './data/image/train_img/'
mask_folder = './data/image/train_mask/'
output_csv = './train_img_info_wtmask.csv'
training_csv='./training_dataset.csv'

train_num_total=10000
pixels_more_100 = []
pixels_less_100=[]
count=0
with open(output_csv, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if int(row['Mask Pixels']) >= 100:
            pixels_more_100.append(row['Filename'])
            count+=1
        else:
            pixels_less_100.append(row['Filename'])

print(count)  #889
pixels_less_100=pixels_less_100[:train_num_total-count]
print(len(pixels_less_100))
image_list=pixels_more_100+pixels_less_100
print(len(image_list))

mask_list=[]
for image in image_list:
    image_idx = image.split("_")[-1]
    mask_list.append(f'train_mask_{image_idx}')

print(len(mask_list))
print(mask_list)

df = pd.DataFrame({'train_img': image_list, 'train_mask': mask_list})

# Save the DataFrame to CSV
df.to_csv(training_csv, index=False)

print("CSV file with image and mask lists created successfully.")


'''
pixels_less_100=pixels_less_100[:]

for files in pixels_less_100:
    file_index=files.split['_'][-1]


case_indices = list(set(item[2] for item in full_dataset))

random.shuffle(case_indices)
split_ratio = 0.8
split_index = int(split_ratio * len(case_indices))

#train_img_1.tif

with open(filtered_output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write header row
    writer.writerow(['Filenames with Mask Pixels >= 100'])
    # Write filtered entries
    for entry in filtered_entries:
        writer.writerow([entry])

print("Filtered CSV file generated successfully.")'''

In [ ]:
'''import pandas as pd
train_df = pd.read_csv('./data/image/train_img_info_wtmask.csv')
train_df.hist('Mask Pixels')
pd.set_option('display.max_rows', None)
total_samples = train_df.shape[0] 
print("Total number of samples in the dataset:", total_samples) #33575
mask_pixel_counts = train_df['Mask Pixels'].value_counts().sort_index()

print(mask_pixel_counts)
filtered_mask_pixel_counts = mask_pixel_counts[(mask_pixel_counts.index >= 3) & (mask_pixel_counts.index <= 200)]

# Plot histogram for filtered data
filtered_mask_pixel_counts.plot(kind='bar', figsize=(10,6), title='Histogram of Mask Pixels')

# Show the plot
plt.show()'''


import pandas as pd

train_df = pd.read_csv('./train_img_info_final.csv')
'''
# Plot histogram with normalized values
train_df['Mask Pixels'].hist(density=True)

# Optionally, set the maximum number of displayed rows
pd.set_option('display.max_rows', None)

train_df['Mask Pixels'].value_counts()'''

